<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Train Practice

## *Data Science Unit 4 Sprint 2 Assignment 3*

Continue to use TensorFlow Keras & a sample of the [Quickdraw dataset](https://github.com/googlecreativelab/quickdraw-dataset) to build a sketch classification model. The dataset has been sampled to only 10 classes and 10000 observations per class. Using your baseline model from yesterday, hyperparameter tune it and report on your highest validation accuracy. Your singular goal today is to achieve the highest accuracy possible.

*Don't forgot to switch to GPU on Colab!*

### Hyperparameters to Tune

At a minimum, tune each of these hyperparameters using any strategy we discussed during lecture today: 
- Optimizer
- Learning Rate
- Activiation Function
  - At least 1 subparameter within the Relu activation function
- Number of Neurons in Hidden Layers
- Number of Hidden Layers
- Weight Initialization

In [3]:
import pandas as pd
import numpy as np
import tensorflow as ts
import seaborn as sns
import shuffle

In [18]:
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

# required for compatibility bewteen sklearn and keras
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle

In [19]:
path="/Users/jenniferquigley/DS22/DS-Unit-4-Sprint-2-Neural-Networks/module2-Train/quickdraw10.npz"

In [20]:
def load_quickdraw10(path):
    # Load dataset
    data = np.load(path)
    
    # Split into X & y
    X = data['arr_0']
    y = data['arr_1']
    
    # normalize X
    X = X.astype('float')/255
    
    # Randomly shuffle X & y
    X, y = shuffle(X, y)
    
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
    
    # Split into train, test sets
    return X_train, y_train, X_test, y_test

In [23]:
X_train, y_train, X_test, y_test = load_quickdraw10(path)

In [36]:
X_train.shape

(67000, 784)

In [35]:
y_train.shape

(67000,)

In [1]:
# Function to create model, required for KerasClassifier
def create_model(units=32):

    # instaniate a Sequential object
    model = Sequential()
    
    # add hidden layer 
    model.add(Dense(units, input_dim=784, activation="relu"))
    
    # add output layer 
    # for multi-class classification, you necessarily must use the softmax act func 
    # number of units = number of unique labels to predict
    model.add(Dense(10, activation = "softmax"))
    
    # complie the model 
    model.compile(loss="sparse_categorical_crossentropy", 
                  optimizer="adam",
                  metrics=["accuracy"])
    
    # return the model 
    return model

In [2]:
model = KerasClassifier(build_fn=create_model)

NameError: name 'KerasClassifier' is not defined

In [39]:
# define the grid search parameters
param_grid = {'batch_size': [32, 64, 128],
              'epochs': [3, 8, 15],
              'units':[32, 64, 98]}

In [41]:
grid = GridSearchCV(estimator=model, 
                    param_grid=param_grid, 
                    n_jobs=-2, 
                    verbose=1, 
                    cv=3)

In [42]:
grid_result = grid.fit(X_train, y_train)

Fitting 3 folds for each of 27 candidates, totalling 81 fits
Epoch 1/15
524/524 [==============================] - 1s 1ms/step - loss: 1.0428 - accuracy: 0.6806
Epoch 2/15
524/524 [==============================] - 1s 1ms/step - loss: 0.6133 - accuracy: 0.8226
Epoch 3/15
524/524 [==============================] - 1s 1ms/step - loss: 0.5291 - accuracy: 0.8464
Epoch 4/15
524/524 [==============================] - 1s 1ms/step - loss: 0.4743 - accuracy: 0.8626
Epoch 5/15
524/524 [==============================] - 1s 1ms/step - loss: 0.4337 - accuracy: 0.8763
Epoch 6/15
524/524 [==============================] - 1s 1ms/step - loss: 0.4132 - accuracy: 0.8811
Epoch 7/15
524/524 [==============================] - 1s 1ms/step - loss: 0.3841 - accuracy: 0.8882
Epoch 8/15
524/524 [==============================] - 1s 1ms/step - loss: 0.3613 - accuracy: 0.8961
Epoch 9/15
524/524 [==============================] - 1s 1ms/step - loss: 0.3327 - accuracy: 0.9034
Epoch 10/15
524/524 [==================

In [44]:
print(f"Best: {grid_result.best_score_} using {grid_result.best_params_}")

Best: 0.851955235004425 using {'batch_size': 128, 'epochs': 15, 'units': 98}


### Stretch Goals
- Implement Bayesian Hyper-parameter Optimization
- Select a new dataset and apply a neural network to it.
- Use a cloud base experiment tracking framework such as weights and biases
- Research potential architecture ideas for this problem. Try Lenet-10 for example. 